In [1]:
%matplotlib inline

In [47]:
from matplotlib import pyplot as plt
import numpy as np
import os
from PIL import Image
import tensorflow as tf
import math
from skimage.transform import resize
import json
from tqdm import tqdm_notebook as tqdm
import random

In [3]:
!bazel build -c opt //im2txt/...

.......
Loading: 
Loading: 0 packages loaded
Analyzing: 17 targets (3 packages loaded)
Analyzing: 17 targets (4 packages loaded)
    currently loading: @local_config_cc//
Analyzing: 17 targets (6 packages loaded)
    currently loading: tools/defaults
INFO: Analysed 17 targets (17 packages loaded).
Building: no action
INFO: Found 17 targets...
Building: no action
[24 / 25] [-----] BazelWorkspaceStatusAction stable-status.txt
INFO: Elapsed time: 1.196s, Critical Path: 0.01s
[25 / 25] no action
INFO: Build completed successfully, 1 total action


In [4]:
import sys
import collections

In [5]:
sys.path.insert(0, 'bazel-bin/im2txt/run_inference.runfiles/im2txt/')

In [6]:
from __future__ import absolute_import
from im2txt import configuration
from im2txt.inference_utils import caption_generator
from im2txt.inference_utils import vocabulary
from im2txt import show_and_tell_model
from im2txt.inference_utils import inference_wrapper_base

In [7]:
tf.logging.set_verbosity(tf.logging.INFO)

In [8]:
ls /hdd/train/im2txt+ssd/3/

checkpoint
eval/
events.out.tfevents.1524214695.aghasy-gpu
graph.pbtxt
model.ckpt-1000000.data-00000-of-00001
model.ckpt-1000000.index
model.ckpt-1000000.meta
model.ckpt-987898.data-00000-of-00001
model.ckpt-987898.index
model.ckpt-987898.meta
model.ckpt-991595.data-00000-of-00001
model.ckpt-991595.index
model.ckpt-991595.meta
model.ckpt-995243.data-00000-of-00001
model.ckpt-995243.index
model.ckpt-995243.meta
model.ckpt-998920.data-00000-of-00001
model.ckpt-998920.index
model.ckpt-998920.meta


In [9]:
checkpoint_path = '/hdd/train/im2txt+ssd-archive/ssd-ext/model.ckpt-591574'

In [10]:
class InferenceWrapper(inference_wrapper_base.InferenceWrapperBase):
    """Model wrapper class for performing inference with a ShowAndTellModel."""

    def __init__(self):
        super(InferenceWrapper, self).__init__()
        self.fetches = {}
        self.post_feed_image = lambda result: 0
    
    def build_model(self, model_config):
        model = show_and_tell_model.ShowAndTellModel(model_config, mode="inference")
        model.build()
        return model
    
    
    
    def feed_image(self, sess, encoded_image):
        self.fetches['initial_state'] = tf.get_default_graph().get_tensor_by_name('lstm/initial_state:0')
        result = sess.run(fetches=self.fetches,
                                 feed_dict={"image_feed:0": encoded_image})
        self.post_feed_image(result)
        return result['initial_state']

    def inference_step(self, sess, input_feed, state_feed):
        softmax_output, state_output = sess.run(
            fetches=["softmax:0", "lstm/state:0"],
            feed_dict={
                "input_feed:0": input_feed,
                "lstm/state_feed:0": state_feed,
            })
        return softmax_output, state_output, None


In [11]:
model = InferenceWrapper()
config = configuration.ModelConfig()
restore_fn = model.build_graph_from_config(config, checkpoint_path)

INFO:tensorflow:Building model.


In [12]:
def test_uninitialized(sess):
    global_vars          = tf.global_variables()
    is_not_initialized   = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]
    if not_initialized_vars:
        print 'Not initalized varibles are: ', [str(i.name) for i in not_initialized_vars]
        # from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
        # print_tensors_in_checkpoint_file(checkpoint_path, all_tensors=True, tensor_name='')
    assert not not_initialized_vars

In [13]:
with tf.Session() as sess:
    restore_fn(sess)
    test_uninitialized(sess)

INFO:tensorflow:Loading model from checkpoint: /hdd/train/im2txt+ssd-archive/ssd-ext/model.ckpt-591574
INFO:tensorflow:Restoring parameters from /hdd/train/im2txt+ssd-archive/ssd-ext/model.ckpt-591574
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-591574


In [14]:
sys.path.insert(0, '..')
sys.path.insert(0, '../slim/')

In [15]:
from object_detection.utils import ops as utils_ops

In [16]:
!cd ..; protoc object_detection/protos/*.proto --python_out=.

In [17]:
sys.path.insert(0, '../object_detection')
from utils import label_map_util
from utils import visualization_utils as vis_util

../object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/usr/local/lib/python2.7/dist-packages/tornado/

In [18]:
PATH_TO_LABELS = os.path.abspath('../object_detection/data/mscoco_label_map.pbtxt')
NUM_CLASSES = 90

In [19]:
label_map  = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [20]:
IMAGE_SIZE = (12, 8)

In [21]:
image_tensor = tf.get_default_graph().get_tensor_by_name('image_feed:0')

In [22]:
def get_detection_tensors():
    ops = tf.get_default_graph().get_operations()
    all_tensor_names = {output.name for op in ops for output in op.outputs}
    tensor_dict = {}
    for key in [
            'num_detections', 'detection_boxes', 'detection_scores',
            'detection_classes', 'detector_image', 'decode/DecodeJpeg'
    ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
            tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                    tensor_name)
    return tensor_dict

In [23]:
def draw_detection_result(output_dict):
    # all outputs are float32 numpy arrays, so convert types as appropriate
    image_np = output_dict['decode/DecodeJpeg']
    # Visualization of the results of a detection.
    image_np = np.array(image_np).astype(np.uint8)[:,:,:3]
#     image_np = resize(image_np, (640, int(640.0 / image_np.shape[0] *image_np.shape[1]), 3))
    vis_util.visualize_boxes_and_labels_on_image_array(
         image_np,
         output_dict['detection_boxes'][0],
         output_dict['detection_classes'][0].astype(np.uint8),
         output_dict['detection_scores'][0],
         category_index,
         instance_masks=None,
         use_normalized_coordinates=True,
         line_thickness=min(image_np.shape[:-1]) / 100)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

In [24]:
# model.fetches = get_detection_tensors()
# model.post_feed_image = draw_detection_result

In [25]:
vocab_file = '/hdd/train/im2txt-mid/word_counts.txt'
vocab = vocabulary.Vocabulary(vocab_file)
generator = caption_generator.CaptionGenerator(model, vocab)

INFO:tensorflow:Initializing vocabulary from file: /hdd/train/im2txt-mid/word_counts.txt
INFO:tensorflow:Created vocabulary with 11520 words


In [26]:
annotation_file = '../../../coco-caption/annotations/captions_val2014.json'
with open(annotation_file) as f:
    js = json.load(f)
images = [(a['id'], a['file_name']) for a in js['images']]

In [27]:
images_dir = '/hdd/datasets/mscoco/dataset/mscoco/val2014/'

In [58]:
random_words_choses = [[vocab.word_to_id(k)] * v for k, v in vocab.vocab.items()]
random_words_choses = [x for x in random_words_choses for i in x]
vocab.end_id in random_words_choses and random_words_choses.remove(vocab.end_id)
vocab.start_id in random_words_choses and random_words_choses.remove(vocab.start_id)

False

In [74]:
new_annotations = []

with tf.Session() as sess:
    restore_fn(sess)
    for image_id, file_name in tqdm(images[:1000]):
        with open(os.path.join(images_dir, file_name), 'r') as content_file:
            content = content_file.read()
        captions = generator.beam_search(sess, content)
        for caption in captions:
            sentence = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]
            sentence = " ".join(sentence)
            sentence = sentence.replace('<S>', '')
            sentence = sentence.replace('  ', ' ').replace('  ', ' ').replace('  ', ' ')
            sentence = sentence.replace('. .', '.').replace('..', '.').replace('. .', '.')
            new_annotations.append(dict(image_id=image_id, caption=sentence, file_name=file_name))
            break
    with open(checkpoint_path + '.captions_result.json', 'w') as f:
        json.dump(new_annotations, f)

INFO:tensorflow:Loading model from checkpoint: /hdd/train/im2txt+ssd-archive/ssd-ext/model.ckpt-591574
INFO:tensorflow:Restoring parameters from /hdd/train/im2txt+ssd-archive/ssd-ext/model.ckpt-591574
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-591574


In [75]:
coco_caption = '../../../coco-caption/'
sys.path.insert(0, coco_caption)
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import matplotlib.pyplot as plt
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
import os
import json
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

In [76]:
# set up file names and pathes
annFile=annotation_file
resFile = checkpoint_path + '.captions_result.json'
evalImgsFile = checkpoint_path + '.captions_evalImgs.json'
evalFile = checkpoint_path + '.captions_eval.json'

# download Stanford models
!{coco_caption}/get_stanford_models.sh

Found Stanford CoreNLP.


In [77]:
# create coco object and cocoRes object
coco = COCO(annFile)
cocoRes = coco.loadRes(resFile)

loading annotations into memory...
0:00:00.497129
creating index...
index created!
Loading and preparing results...     
DONE (t=0.03s)
creating index...
index created!


In [78]:
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)

# evaluate on a subset of images by setting
# cocoEval.params['image_id'] = cocoRes.getImgIds()
# please remove this line when evaluating the full validation set
cocoEval.params['image_id'] = cocoRes.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
cocoEval.evaluate()

tokenization...
setting up scorers...
computing Bleu score...
{'reflen': 10371, 'guess': [10371, 9371, 8371, 7371], 'testlen': 10371, 'correct': [10371, 1532, 139, 17]}
ratio: 1.0
Bleu_1: 1.000
Bleu_2: 0.404
Bleu_3: 0.139
Bleu_4: 0.050
computing METEOR score...
METEOR: 0.415
computing Rouge score...
ROUGE_L: 0.469
computing CIDEr score...
CIDEr: 1.102
computing SPICE score...
SPICE: 0.180


In [79]:
# print output evaluation scores
for metric, score in cocoEval.eval.items():
    print '%s: %.3f'%(metric, score)

CIDEr: 1.102
Bleu_4: 0.050
Bleu_3: 0.139
Bleu_2: 0.404
Bleu_1: 1.000
ROUGE_L: 0.469
METEOR: 0.415
SPICE: 0.180


In [80]:
# demo how to use evalImgs to retrieve low score result
evals = [eva for eva in cocoEval.evalImgs if eva['CIDEr']<30]
print 'ground truth captions'
imgId = evals[0]['image_id']
annIds = coco.getAnnIds(imgIds=imgId)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

print '\n'
print 'generated caption (CIDEr score %0.1f)'%(evals[0]['CIDEr'])
annIds = cocoRes.getAnnIds(imgIds=imgId)
anns = cocoRes.loadAnns(annIds)
coco.showAnns(anns)

img = coco.loadImgs(imgId)[0]
I = io.imread('%s/images/%s/%s'%('./','val2014',img['file_name']))
plt.imshow(I)
plt.axis('off')
plt.show()

ground truth captions
A train traveling down tracks next to lights.
A blue and silver train next to train station and trees.
A blue train is next to a sidewalk on the rails.
A passenger train pulls into a train station.
A train coming down the tracks arriving at a station.


generated caption (CIDEr score 1.1)
lights. traveling next to tracks A down train


IOError: [Errno 2] No such file or directory: u'.//images/val2014/COCO_val2014_000000184321.jpg'

In [ ]:
# plot score histogram
ciderScores = [eva['CIDEr'] for eva in cocoEval.evalImgs]
plt.hist(ciderScores)
plt.title('Histogram of CIDEr Scores', fontsize=20)
plt.xlabel('CIDEr score', fontsize=20)
plt.ylabel('result counts', fontsize=20)
plt.show()

In [ ]:
# save evaluation results to ./results folder
json.dump(cocoEval.evalImgs, open(evalImgsFile, 'w'))
json.dump(cocoEval.eval,     open(evalFile, 'w'))

In [ ]:
js['info']